In [ ]:
import requests
import json
import pandas as pd
import time
import re
pd.set_option('display.max_colwidth', None)

In [ ]:
def get_tx_hash(events):
    for event in events:
        if(event['type'] == 'tx'):
            for attr in event['attributes']:
                if('composite_key' in attr and attr['composite_key']=='tx.hash'):
                    return attr['value']
    return None

In [ ]:
def get_tx_action(events):
    for event in events:
        if(event['type'] == 'message'):
            for attr in event['attributes']:
                if('key' in attr and attr['key']=='action'):
                    return attr['value']
    return None

In [ ]:
def get_tx_contract_address(events):
    for event in events:
        if(event['type'] == 'execute'):
            action = None
            for attr in event['attributes']:
                if('key' in attr and attr['key']=='_contract_address'):
                    return attr['value']
    return None

In [ ]:
def get_tx_contract_action(events):
    for event in events:
        if(event['type'] == 'wasm'):
            for attr in event['attributes']:
                if('key' in attr and attr['key']=='action'):
                    return attr['value']
    return None

In [ ]:
def get_tx_coin_spent(event):
    spender = None
    amounts = []
    if(event['type'] == 'coin_spent'):    
        for attr in event['attributes']:
            if('key' in attr and attr['key']=='spender'):
                spender = attr['value']
            if('key' in attr and attr['key']=='amount'):
                amounts = attr['value'].split(',')
    return spender, amounts

In [ ]:
def get_tx_coin_received(event):
    receiver = None
    amounts = []
    if(event['type'] == 'coin_received'):
        for attr in event['attributes']:
            if('key' in attr and attr['key']=='receiver'):
                receiver = attr['value']
            if('key' in attr and attr['key']=='amount'):
                amounts = attr['value'].split(',')
    return receiver, amounts

In [ ]:
def parse_tx(tx):
    raw_msg = tx
    tx_parsed_tuples = []
    tx_created_at = tx['block']['created_at']
    tx_events = tx['events']
    tx_hash = get_tx_hash(tx_events)
    tx_action = get_tx_action(tx_events)
    tx_contract_address = get_tx_contract_address(tx_events)
    tx_contract_action = get_tx_contract_action(tx_events)
    for event in tx_events:
        spender, amounts = get_tx_coin_spent(event)
        if(spender and amounts and len(amounts) > 0):
            for amount in amounts:
                tx_parsed_tuples.append((tx_created_at, tx_hash, spender, amount, event['type'], tx_action, tx_contract_address, tx_contract_action))
        receiver, amounts = get_tx_coin_received(event)
        if(receiver and amounts and len(amounts) > 0):
            for amount in amounts:
                tx_parsed_tuples.append((tx_created_at, tx_hash, receiver, amount, event['type'], tx_action, tx_contract_address, tx_contract_action))
    return tx_parsed_tuples

In [ ]:
df = None
i = 0
limit = 10
raw_msgs = []
while True:
    # API endpoint URL
    address = 'kujira1pkmjc7p7lqg6m3nm7csde88pnpad945kla5hyu'
    url = f"https://api.kujira.app/api/txs?q={address}&limit={limit}&offset={i}&order_by=rowid&order_dir=desc"
    print(url)
    # Send a GET request to the API
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Retrieve the JSON data
        n_txs = len(response.json()['txs'])
        print(f"Received: {n_txs} txs")
        if(n_txs == 0):
            break
        
        json_data = response.json()
        raw_msgs.append(json_data)
        vals = []
        for tx in response.json()['txs']:
            # Call the parse_json function to parse the JSON data
            vals = vals + parse_tx(tx)
        if(df is None):
            df = pd.DataFrame(vals, columns=['created_at','tx_hash', 'address', 'amount','operation','action', 'contract_address', 'contract_action'])
        else:
            df = pd.concat([df, pd.DataFrame(vals, columns=['created_at','tx_hash', 'address', 'amount','operation','action', 'contract_address', 'contract_action'])])
        i += limit
        print(i)
        print(len(df))
    else:
        # Handle any errors that occurred during the request
        print("Error:", response.status_code)
df.to_csv(f'data/kujira/{address}.csv', index=False)

In [ ]:
df.created_at = pd.to_datetime(df.created_at)

In [ ]:
df.created_at.max()

In [ ]:
df.head()

In [ ]:
response = requests.get('https://api.rorcualnodes.com/contracts')

In [ ]:
data = response.json()

In [ ]:
with open(f'data/kujira/contracts.json', "w") as json_file:
    json.dump(data, json_file)